## Calculate a Loan Instalment Schedule 2
Given start-date, principle, number of instalment and interest rate (per year):

* Calculates the EMI (equal monthly instalment) amount 
* Calculates the instalment amortisation schedule
* Maps to a DBEI (optimised) loan in Mambu with Days-in-year: actual/365
* Also supports calculating Mambu's DBEI (non-optimised) with Days-in-year: actual/365
    * NOTE: This uses an excel PMT calculation to work out EMI and adds extras on the last instalment



In [274]:
import sympy as sp
import sys
import datetime
from dateutil import parser
import calendar
# Need to increase the recusion limit (else .subs fails for 300 instalments)
sys.setrecursionlimit(5000)

In [325]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day)

def daysInPeriod(startDate):
    if isinstance(startDate,datetime.date):
        dt = startDate
    else:
        dt = parser.parse(startDate).date()
    dt2 = add_months(dt, 1)
    daysInPeriod = 0
    daysInPeriod = dt2 - dt
    return (dt,dt2,daysInPeriod.days)

def monthlyPeriodInterest(days, interestPerYearPercent, daysCalcMethod):
    if daysCalcMethod == "30/360":
        periodInterest = interestPerYearPercent / 1200
    elif daysCalcMethod == "actual/365":
        periodInterest = days * (interestPerYearPercent / 36500)
    else:
        assert True == False, "Unknown daysCalcMethod: {0}".format(daysCalcMethod)
        
    return periodInterest

def daysInPeriod2(startDate, interestPerYearPercent, daysCalcMethod):
    dateStuff = daysInPeriod(startDate)
    periodInterest = monthlyPeriodInterest(dateStuff[2], interestPerYearPercent, daysCalcMethod)
    # classic python trick here if tuple only has 1 element - add ,
    dateStuff = dateStuff + (periodInterest,)
    return dateStuff
        


In [328]:
daysInPeriod2("01 Feb 2019",12,'actual/365')

(datetime.date(2019, 2, 1),
 datetime.date(2019, 3, 1),
 28,
 0.009205479452054795)

In [329]:
def schedule(startDate, numInstalments, annualInterest, daysCalcMethod):
    DAYS_CALC_METHOD = daysCalcMethod # See monthlyPeriodInterest above for supported daysCalcMethod
    P, r, n, E = sp.symbols('P r n E')
    dateStuff = daysInPeriod2(startDate, annualInterest, DAYS_CALC_METHOD)
    instalmentList = [{'num':1,'from_date':dateStuff[0], 'days_in_period': dateStuff[2],
                       'interest_expected':P*dateStuff[3],'principle_remaining':P*(1+dateStuff[3])-E}]
    dateStuff = daysInPeriod2(dateStuff[1], annualInterest, DAYS_CALC_METHOD)
    monthlyPeriodInterest(dateStuff[2],annualInterest,'30/360')
    for i in range(1,numInstalments):
        instObj = {}
        instObj["num"] = i+1
        instObj["interest_expected"] = instalmentList[i-1]['principle_remaining'] * dateStuff[3]
        instObj["principle_remaining"] = instalmentList[i-1]['principle_remaining']*(1+dateStuff[3])-E
        instObj["from_date"] = dateStuff[0]
        instObj["days_in_period"] = dateStuff[2]
        instalmentList.append(instObj)
        dateStuff = daysInPeriod2(dateStuff[1], annualInterest, DAYS_CALC_METHOD)
    return instalmentList

In [363]:
def expandSchedule(startDate, OrigPrinciple, annualInterest, numInstalments, daysCalcMethod, daysCalcMethodEMI=None):
    ROUND_NUMDIGITS = 10
    shList = schedule(startDate, numInstalments, annualInterest, daysCalcMethod)
    
    # Solve the Equation for E - for the last instalment
    if daysCalcMethodEMI is None:
        prin = shList[numInstalments-1]['principle_remaining']
        expandedPrin = prin.subs(P,OrigPrinciple)
        equalMonthlyInstalment = sp.solve(expandedPrin, E)[0]
    else:
        # This is to simulate Mambu's standard DBEI (i.e. without optimised setting)
        # Mambu calculates the EMI using standard excel PMT style equation - i.e. daysCalcMethodEMI=="30/360"
        shList2 = schedule(startDate, numInstalments, annualInterest, daysCalcMethodEMI)
        prin = shList2[numInstalments-1]['principle_remaining']
        expandedPrin = prin.subs(P,OrigPrinciple)
        equalMonthlyInstalment = sp.solve(expandedPrin, E)[0]
    
    # Now gothrough the complete shList and plug in values for all the variables
    for i in range(numInstalments):
        instObj = shList[i]
        instObj["interest_expected"] = round(instObj["interest_expected"].subs(
            {
                P:OrigPrinciple,
                E:equalMonthlyInstalment
            }), ROUND_NUMDIGITS)
        instObj["principle_remaining"] = round(instObj["principle_remaining"].subs(
            {
                P:OrigPrinciple,
                E:equalMonthlyInstalment
            }), ROUND_NUMDIGITS)
        
        if i == 0:
            previousPrinciple = OrigPrinciple
        else:
            previousPrinciple = shList[i-1]["principle_remaining"]
        instObj["principle_expected"] = previousPrinciple - instObj["principle_remaining"]
        
        instObj["total_expected"] = instObj["principle_expected"] + instObj["interest_expected"]

    # Make sure we have fully paid off the loan in the last instalment   
    # There may be some extras to pay when using Mambu's standard DBEI calc
    lastInstalmentObj = shList[numInstalments-1]
    lastInstalmentObj['principle_expected'] = lastInstalmentObj['principle_expected'] + lastInstalmentObj['principle_remaining']
    lastInstalmentObj["total_expected"] = lastInstalmentObj["principle_expected"] + lastInstalmentObj["interest_expected"]
    return (equalMonthlyInstalment,shList)

In [365]:
#expandSchedule("16 Dec 2019", OrigPrinciple=5000, annualInterest=12, numInstalments=5, daysCalcMethod="actual/365", daysCalcMethodEMI="30/360" )
expandSchedule("16 Dec 2019", OrigPrinciple=5000, annualInterest=12, numInstalments=5, daysCalcMethod="actual/365")
#expandSchedule("01 Dec 2019", OrigPrinciple=5000, annualInterest=12, numInstalments=5, daysCalcMethod="30/360" )

(1030.31769900205,
 [{'num': 1,
   'from_date': datetime.date(2019, 12, 16),
   'days_in_period': 31,
   'interest_expected': 50.9589041096,
   'principle_remaining': 4020.6412051075,
   'principle_expected': 979.35879489250,
   'total_expected': 1030.3176990021},
  {'num': 2,
   'interest_expected': 40.9774939260,
   'principle_remaining': 3031.3010000315,
   'from_date': datetime.date(2020, 1, 16),
   'days_in_period': 31,
   'principle_expected': 989.34020507600,
   'total_expected': 1030.3176990020},
  {'num': 3,
   'interest_expected': 28.9011711784,
   'principle_remaining': 2029.8844722078,
   'from_date': datetime.date(2020, 2, 16),
   'days_in_period': 29,
   'principle_expected': 1001.4165278237,
   'total_expected': 1030.3176990021},
  {'num': 4,
   'interest_expected': 20.6881376346,
   'principle_remaining': 1020.2549108403,
   'from_date': datetime.date(2020, 3, 16),
   'days_in_period': 31,
   'principle_expected': 1009.6295613675,
   'total_expected': 1030.3176990021},
